In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,make_column_selector

In [49]:
d=pd.read_csv('C:/Users\ESHITA\Downloads/accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')

C:\Users\ESHITA\AppData\Local\Temp\ipykernel_30328\3225256400.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  d=pd.read_csv('C:/Users\ESHITA\Downloads/accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')


In [50]:
data=d[["loan_amnt","term","int_rate", "installment","grade","sub_grade","fico_range_low","annual_inc","dti","emp_length","purpose","open_acc","total_acc","revol_util","delinq_2yrs",
"inq_last_6mths","loan_status"]]
print(data.head())

   loan_amnt        term  int_rate  installment grade sub_grade  \
0     3600.0   36 months     13.99       123.03     C        C4   
1    24700.0   36 months     11.99       820.28     C        C1   
2    20000.0   60 months     10.78       432.66     B        B4   
3    35000.0   60 months     14.85       829.90     C        C5   
4    10400.0   60 months     22.45       289.91     F        F1   

   fico_range_low  annual_inc    dti emp_length             purpose  open_acc  \
0           675.0     55000.0   5.91  10+ years  debt_consolidation       7.0   
1           715.0     65000.0  16.06  10+ years      small_business      22.0   
2           695.0     63000.0  10.78  10+ years    home_improvement       6.0   
3           785.0    110000.0  17.06  10+ years  debt_consolidation      13.0   
4           695.0    104433.0  25.37    3 years      major_purchase      12.0   

   total_acc  revol_util  delinq_2yrs  inq_last_6mths loan_status  
0       13.0        29.7          0.0     

In [51]:
X=data.drop('loan_status',axis=1)
y=data['loan_status']

In [52]:
num_pipe=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
cat_pipe=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore',
                            sparse_output=True,
                            dtype=int))
])
ct=ColumnTransformer( transformers=[
        ('num', num_pipe,
         make_column_selector(dtype_include=(int, float))),
        ('cat', cat_pipe,
         make_column_selector(dtype_include=object))
    ])

In [53]:
X_new=ct.fit_transform(X)

In [54]:
y_new= (data['loan_status']=='Fully Paid').astype(int)

In [55]:
X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.2,random_state=42)

In [56]:
xgb=XGBClassifier(
    n_estimators=120,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
)
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None,
              num_parallel_tree=None, ...)

In [57]:
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
y_prob=xgb.predict_proba(X_test)
y_pred=(y_prob[:,1]>0.4).astype(int)
print(classification_report(y_test,y_pred))
roc_auc_score(y_test, y_prob[:,1])

              precision    recall  f1-score   support

           0       0.79      0.57      0.66    237500
           1       0.64      0.84      0.72    214641

    accuracy                           0.70    452141
   macro avg       0.72      0.70      0.69    452141
weighted avg       0.72      0.70      0.69    452141



0.8004324155168274

In [58]:
train_acc = accuracy_score(y_train, xgb.predict(X_train))
test_acc  = accuracy_score(y_test, xgb.predict(X_test))
print(train_acc)
print(test_acc)

0.7317445923828902
0.7312188012146653


In [59]:
y_train.value_counts(normalize=True)


loan_status
0    0.523317
1    0.476683
Name: proportion, dtype: float64

The XGBoost classifier achieved the best performance among all models, demonstrating strong generalization with nearly identical training and test scores. The model significantly outperformed the baseline, indicating that it successfully learned meaningful patterns from the data without overfitting. ROC-AUC was used as the primary evaluation metric, making the model suitable for credit risk prediction where ranking defaulters is critical.